### Generate chats between users with the following scenarios 

- Technical 
    - want to brainstorm approach
    - facing issues with parts of the codebase that the team member has more context on
    - brain scratching technical issues like database migration gone bad or unknown merge conflicts
    - production hot fixes

In [25]:
from ollama import chat
from pydantic import BaseModel, Field

In [ ]:
class ChatMessage(BaseModel):
    user: str
    message: str

class MessageStream(BaseModel):
    messages: list[ChatMessage]


purpose = "ask for help to brainstorm for a made up technical problem."
intent = "setup a quick sync up call"

def generate_chat(purpose, intent, model="qwen2.5-coder:32b", retries_left=3):
    prompt = f"""
    I need your help to generate dummpy data for a demo.
    create a sequence of messages similar to a chat in an IRC channel with multiple users where,

    the user wants to {intent} with the purpose of {purpose}.
    
    
    1. pick the number of users and their usernames in the conversation
    2. generate replies with other users participating to help in response to continue the chat for 3-4 messages or more
     
    Example of what an IIRC conversation might look like: 

    hstefan 	hey e_t_, been wrestling with some virtualbox resolution issues
    e_t_ 	hstefan: yeah? want to jump on a call and troubleshoot together?
    flavio 	I could join if you’re doing a screen share debugging session
    hstefan 	that sounds great - google meet or discord?
    e_t_ 	discord works for me. how about this evening around 8?
    flavio 	+1 for discord, 8pm UTC?
    hstefan 	works for me. I’ll send the invite link
    e_t_ 	cool, see you all then

    DO NOT use the same conversation as the example, we need variety.
    """
    if retries_left == 0:
        raise ValueError("Failed to generate chat after retries")
    response = chat(
        messages = [
            {'role': 'user', 'content': prompt},
        ],
        model=model,
        format=MessageStream.model_json_schema()
    )
    try:
        return MessageStream.model_validate_json(response.message.content)
    except Exception as e:
        print(f"Error generating response: {e}, for purpose: {purpose}")
        return generate_chat(prompt, retries_left - 1)


In [50]:
generate_chat("discussing technical debt", "request for an immediate call")

MessageStream(messages=[ChatMessage(user='tech_lead', message='Hey team, been dealing with a pile of technical debt lately. Could we get an immediate call to discuss some strategies?', calendar_event=False), ChatMessage(user='dev1', message='Absolutely! What platform are you thinking for the call?', calendar_event=False), ChatMessage(user='tech_lead', message='Zoom works fine with me. How about right now, if everyone is free?', calendar_event=True), ChatMessage(user='dev2', message='+1 for Zoom and immediately sounds good to me.', calendar_event=False), ChatMessage(user='qa_engineer', message='Sure, I’m in. Let’s do this now!', calendar_event=False)])

In [51]:
generate_chat("discussing technical debt", "request for an immediate call", model="deepseek-r1:32b")

MessageStream(messages=[ChatMessage(user='e_', message="Hey everyone, I think we need to address our technical debt. It's piling up and causing some serious issues in our project.", calendar_event=False), ChatMessage(user='hstefan', message='Agreed, e_. The UI is getting really laggy because of it.', calendar_event=False), ChatMessage(user='e_', message="Alright, I want to set up a call right away. Let's discuss how we can tackle this. Who's available?", calendar_event=True), ChatMessage(user='flavio', message="I'm free now. Let's jump on it.", calendar_event=False), ChatMessage(user='hstefan', message='I can join in 15 minutes. How about we start then?', calendar_event=True), ChatMessage(user='e_', message="Great! Let's connect via Zoom. I'll send the link shortly.", calendar_event=False)])

Assumptions: all of the messages generated here all lead to a calendar event, let's generate 10 chats per scenario this is not 100% true but for the assignment

- Technical 
    - want to brainstorm approach
    - facing issues with parts of the codebase that the team member has more context on
    - brain scratching technical issues like database migration gone bad or unknown merge conflicts
    - production hot fixes
- Team operations proposal
    - sprint plannng discussion
    - feedback discussions (one-on-one)
    - roadmap plannning
- Client meeting proposal
    - feedback on a presentation
    - how to showcase particular edge cases being handled
    - dry run of demos
    - team lead proposes a meeting after a client call to discuss next steps

In [67]:
from itertools import product

intents = [
    "get someone on a quick call to help out",
    "plan a meeting",
    "reschedule an existing meeting"
]

purposes = [
    "brainstorm a system design",
    "brainstorm on an issue you are stuck on",
    "facing issues with parts of the codebase that the team member has more context on",
    "brain scratching technical issues like database migration gone bad or unknown merge conflicts", 
    "production hot fixes", 
    "sprint plannng discussion", 
    "feedback discussions (one-on-one)", 
    "roadmap plannning", 
    "feedback on a presentation", 
    "how to showcase particular edge cases being handled for an update to the client",
    "dry run of demos to showcase to the client",
    "team lead proposes a meeting after a client call to discuss next steps"
]

def generate_sample_chat_messages(purposes: list[str], intents: list[str], n: int):
    for intent, purpose in product(intents, purposes):
        for _ in range(n):
            yield generate_chat(
                    purpose=purpose,
                    intent=intent,
                    model="deepseek-r1:32b"
                )
   

In [70]:
data = generate_sample_chat_messages(purposes[:1], intents[:1], 1)
sample = data.__next__()

In [74]:
sample.model_dump_json()

'{"messages":[{"user":"alice","message":"Hey everyone, I\'m stuck on this system design problem. Does anyone have a free moment to brainstorm with me?"},{"user":"bob","message":"Sure, alice! What\'s the problem about?"},{"user":"charlie","message":"I can help too. Maybe we should set up a quick call for it."},{"user":"alice","message":"Great! How about joining via Zoom? I think that\'s widely available."},{"user":"bob","message":"Zoom works for me. When are you thinking?"},{"user":"charlie","message":"I\'m free anytime after 3 PM today."},{"user":"alice","message":"Perfect! Let\'s meet at 3:30 PM then. I\'ll send the link shortly."}]}'

The data is not accurate but the idea is that it generates enough bag of words that can represent calendar event sentences